**Connect google drive**

In [1]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Mon Nov 20 10:06:24 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
'60 day workout challenge.pdf'	 Dog_Detection_35_Classes	  'Nintendo Account.gdoc'   Videos
'Colab Notebooks'		'Felhasználói dokumentáció.docx'   ticket.pdf
'Dj kex works'			'Kóspallag 2022'		   Trio-Wallpaper.png


**1) Clone, configure & compile Darknet**



In [3]:
# Clone
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15833, done.
remote: Counting objects: 100% (294/294), done.
remote: Compressing objects: 100% (152/152), done.
remote: Total 15833 (delta 157), reused 219 (delta 140), pack-reused 15539
Receiving objects: 100% (15833/15833), 14.76 MiB | 16.66 MiB/s, done.
Resolving deltas: 100% (10587/10587), done.


In [4]:
# Configure
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

/content/darknet


In [5]:
# Compile
!make

mkdir -p ./obj/
mkdir -p backup
mkdir -p results
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -rdynamic -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:945:23: warning: variable ‘rgb’ set but not used []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wunused-but-set-variable-Wunused-but-set-variable]8;;]
  945 |                 float rgb[3];
      |                       ^~~
./src/image_opencv.cpp: In function ‘void cv_draw_object(image, float*, int, int, int*, float*, int*, int, char**)’:
./src/image_opencv.cpp:1443:14: warning: unused variable ‘bu

**2) Configure yolov3.cfg file**

In [6]:
# Make a copy of yolov3.cfg
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [7]:
# Change lines in yolov3.cfg file
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 70000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=35@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=35@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=35@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=120@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=120@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=120@' cfg/yolov3_training.cfg

**3) Create .names and .data files**

In [8]:
!echo -e 'Afgan Hound\nAustralian Terrier\nBorder Collie\nBouvier des Flandres\nClumber Spaniel\nCollie\nDalmatian\nDoberman\nEnglish Springer\nWire Fox Terrier\nFrench Bulldog\nGerman Shepherd\nGiant Schnauzer\nGolden Retiver\nIrish Setter\nKomondor\nKuvasz\nMaltese Dog\nOld English Sheepdog\nPapillon\nPomeranian\nPug\nRottweiler\nSaluki\nSchipperke\nScottish Deerhund\nSiberian Husky\nSoft Coated Wheaten Terrier\nStandard Poodle\nTibetan Terrier\nTibetan Mastiff\nVizsla\nWelsh Springer Spaniel\nWhippet\nYorkshire Terrier' > data/obj.names
!echo -e 'classes= 35\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/Dog_Detection_35_Classes/weights' > data/obj.data

**4) Save yolov3_training.cfg and obj.names files in Google drive**

In [ ]:
!cp cfg/yolov3_training.cfg /mydrive/Dog_Detection_35_Classes/yolov3_testing.cfg
!cp data/obj.names /mydrive/Dog_Detection_35_Classes/classes.txt

**5) Create a folder and unzip image dataset**

In [9]:
!mkdir data/obj
!unzip /mydrive/Dog_Detection_35_Classes/images.zip -d data/obj

A streamkimeneten csak az utolsó 5000 sor látható.
  inflating: data/obj/n02110958_327.JPEG  
  inflating: data/obj/n02110958_327.txt  
  inflating: data/obj/n02110958_3284.JPEG  
  inflating: data/obj/n02110958_3284.txt  
  inflating: data/obj/n02110958_329.JPEG  
  inflating: data/obj/n02110958_329.txt  
  inflating: data/obj/n02110958_330.JPEG  
  inflating: data/obj/n02110958_330.txt  
  inflating: data/obj/n02110958_3302.JPEG  
  inflating: data/obj/n02110958_3302.txt  
  inflating: data/obj/n02110958_3304.JPEG  
  inflating: data/obj/n02110958_3304.txt  
  inflating: data/obj/n02110958_331.JPEG  
  inflating: data/obj/n02110958_331.txt  
  inflating: data/obj/n02110958_333.JPEG  
  inflating: data/obj/n02110958_333.txt  
  inflating: data/obj/n02110958_3338.JPEG  
  inflating: data/obj/n02110958_3338.txt  
  inflating: data/obj/n02110958_3343.JPEG  
  inflating: data/obj/n02110958_3343.txt  
  inflating: data/obj/n02110958_335.JPEG  
  inflating: data/obj/n02110958_335.txt  
  in

**6) Copy train.txt and test.txt files**

In [10]:
!cp /mydrive/Dog_Detection_35_Classes/train.txt data/train.txt
!cp /mydrive/Dog_Detection_35_Classes/test.txt data/test.txt

**7) Download pre-trained weights for the convolutional layers file**

In [ ]:
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2023-11-13 21:26:51--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  95.5MB/s    in 1.6s    

2023-11-13 21:26:52 (95.5 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**8) Start training**

In [ ]:
#!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show
# Uncomment below and comment above to re-start your training from last saved weights
!./darknet detector train data/obj.data cfg/yolov3_training.cfg /mydrive/Dog_Detection_35_Classes/weights/yolov3_training_last.weights -dont_show

A streamkimeneten csak az utolsó 5000 sor látható.
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
 total_bbox = 52643, rewritten_bbox = 0.045590 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.857620), count: 6, class_loss = 0.548615, iou_loss = 0.209399, total_loss = 0.758014 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.541024), count: 1, class_loss = 0.292206, iou_loss = 0.297244, total_loss = 0.589450 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
 total_bbox = 52650, rewritten_bbox = 0.045584 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.863503), count: 4, class_loss = 0.322235, iou_loss = 0.130467, total_loss = 0.452701 
v3 (mse

**9) Test Model**

In [ ]:
!./darknet detector map data/obj.data cfg/yolov3_training.cfg /mydrive/Dog_Detection_35_Classes/weights/yolov3_training_last.weights